In [126]:
import numpy
written_train =numpy.load("written_train(1).npy")
spoken_train =numpy.load("spoken_train(1).npy")
match_train = numpy.load("match_train(1).npy")
feature_amount = spoken_train[0].shape[1]
print("shape written:", written_train.shape)
print("shape spoken", spoken_train.shape)
print("shape spoken indiv:", spoken_train[4].shape)
print(feature_amount)
print(match_train)

shape written: (45000, 784)
shape spoken (45000,)
shape spoken indiv: (38, 13)
13
[False False False ... False False False]


In [127]:

import pandas as pd
from numpy import array

maxs = []
for i in range(0, len(spoken_train)):
    maxs.append(spoken_train[i].shape[0])
maxlen_spoken_train = max(maxs)

from keras.preprocessing.sequence import pad_sequences

spoken_test_3d= pad_sequences(spoken_train, maxlen= max(maxs))
print(spoken_train_3d.shape)

spoken_train_2d =spoken_train_3d.reshape(45000, maxlen_spoken_train*feature_amount)

spoken_train_2d



    

(45000, 93, 13)


array([[ 0,  0,  0, ..., -1,  0,  0],
       [ 0,  0,  0, ..., -2,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., -1,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [128]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
spoken_train_mmsc = mm_scaler.fit_transform(spoken_train_2d)
written_train_mmsc = mm_scaler.fit_transform(written_train)

C:\Users\user2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [129]:
X_train = numpy.hstack([spoken_train_mmsc, written_train_mmsc])
print(X_train.shape)
y_train=match_train

(45000, 1993)


In [131]:
from imblearn.over_sampling import SMOTE

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==0)))
print("Before OverSampling, counts of label '2': {} ".format(sum(y_train==1)))

sm = SMOTE(random_state=4)
X_res, y_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of X: {}'.format(X_res.shape))
print('After OverSampling, the shape of y: {} \n'.format(y_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_res==0)))
print("After OverSampling, counts of label '2': {}".format(sum(y_res==1)))


X= numpy.array(X_res)
y= numpy.array(y_res)


Before OverSampling, counts of label '1': 40461
Before OverSampling, counts of label '2': 4539 
[[0.         0.         0.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 0.875      0.875      0.625     ]
 [0.         0.         0.         ... 1.         1.         1.        ]
 ...
 [0.         0.         0.         ... 1.         0.97146594 0.88451135]
 [0.         0.         0.         ... 0.81456298 0.96771851 1.        ]
 [0.         0.         0.         ... 0.87116476 1.         0.80674715]]
After OverSampling, the shape of X: (80922, 1993)
After OverSampling, the shape of y: (80922,) 

After OverSampling, counts of label '1': 40461
After OverSampling, counts of label '2': 40461


In [143]:
X_train=X
y_train=y
print(y_train.shape)

(80922,)


In [146]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# instantiate labelencoder object
le = LabelEncoder()
y_train = le.fit_transform(y)
print(y_train)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = y_train.reshape(len(y_train), 1)
y_train = onehot_encoder.fit_transform(integer_encoded)
print(y_train)

[0 0 0 ... 1 1 1]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


C:\Users\user2\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


# LSTM

In [147]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [148]:
 print(X_train.shape)
X_train = X_train.reshape(80922,1,1993)


(80922, 1993)


In [149]:
print(X_train.shape)
print(y_train.shape)

(80922, 1, 1993)
(80922, 2)


In [150]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

#### LSTM MODEL
model = Sequential()
model.add(LSTM(32, return_sequences=False, input_shape=(1,1993)))
#model.add(Dense(32, input_dim=784))
model.add(Dense((2)))

#### EARLY STOPPING AND CALLBACK
#callbacks_list = [
#    keras.callbacks.ModelCheckpoint(
 #       filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
#        monitor='val_loss', save_best_only=True),
 #       keras.callbacks.EarlyStopping(monitor='acc', patience=1)]

#keras.callbacks.EarlyStopping(monitor='acc', patience=1)]
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10, batch_size=500, verbose=2,
        validation_split=0.33)

Train on 54217 samples, validate on 26705 samples
Epoch 1/10
 - 6s - loss: 0.5882 - acc: 0.2587 - val_loss: 1.3790 - val_acc: 1.0000
Epoch 2/10
 - 2s - loss: 0.5664 - acc: 0.2537 - val_loss: 1.3774 - val_acc: 1.0000
Epoch 3/10
 - 2s - loss: 0.5664 - acc: 0.2537 - val_loss: 1.3804 - val_acc: 1.0000
Epoch 4/10
 - 3s - loss: 0.5665 - acc: 0.2537 - val_loss: 1.3756 - val_acc: 1.0000
Epoch 5/10
 - 3s - loss: 0.5664 - acc: 0.2537 - val_loss: 1.3491 - val_acc: 1.0000
Epoch 6/10
 - 3s - loss: 0.5665 - acc: 0.2537 - val_loss: 1.3783 - val_acc: 1.0000
Epoch 7/10
 - 3s - loss: 0.5665 - acc: 0.2537 - val_loss: 1.4089 - val_acc: 1.0000
Epoch 8/10
 - 3s - loss: 0.5665 - acc: 0.2537 - val_loss: 1.3768 - val_acc: 1.0000
Epoch 9/10
 - 3s - loss: 0.5664 - acc: 0.2537 - val_loss: 1.3773 - val_acc: 1.0000
Epoch 10/10
 - 3s - loss: 0.5664 - acc: 0.2537 - val_loss: 1.3949 - val_acc: 1.0000


# TESTEN

In [101]:
written_test =numpy.load("written_test(1).npy")
spoken_test =numpy.load("spoken_test(1).npy")

In [102]:
import pandas as pd
from numpy import array

maxs = []
for i in range(0, len(spoken_test)):
    maxs.append(spoken_test[i].shape[0])
maxlen_spoken_test = max(maxs)

from keras.preprocessing.sequence import pad_sequences

spoken_test_3d= pad_sequences(spoken_test, maxlen= max(maxs))
print(spoken_test_3d.shape)

spoken_test_2d =spoken_test_3d.reshape(15000, maxlen_spoken_test*feature_amount)

spoken_test_2d


(15000, 93, 13)


array([[ 0,  0,  0, ..., -1,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., -2,  0,  1],
       ...,
       [ 0,  0,  0, ..., -2, -1,  0],
       [ 0,  0,  0, ..., -2, -1,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [118]:
mm_scaler = MinMaxScaler()
spoken_test_mmsc = mm_scaler.fit_transform(spoken_test_2d)
written_test_mmsc = mm_scaler.fit_transform(written_test)


C:\Users\user2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [117]:
X_test = numpy.hstack([spoken_test_mmsc, written_test_mmsc])
X_test = X_test.reshape(15000,1,1993)

In [119]:
from sklearn.metrics import accuracy_score

y_pred = model.predict_classes(X_test, verbose=1)

print(y_pred)

print(1 in match_train)

15000/15000 [==============================] - 1s 42us/step
[0 0 0 ... 0 0 0]
True


In [115]:
numpy.unique(match_train, return_counts=True)

(array([False,  True]), array([40461,  4539], dtype=int64))